In [1]:
# Import all required libraries
import pandas
from pandas import *
import numpy
from datetime import datetime

# Initialize values

env = None
sample_size = None
# Set env, if env = test, will only be run locally and display the result
env = "prod"
env = "test"

# Number of value on which to train, if null, train on all value
sample_size = 40000

In [2]:
# Read training data + test data
df_data = pandas.read_csv("../input/train.csv")
df_test = pandas.read_csv("../input/test.csv")


# Display basic information
display(df_data.head(5))
print(df_data.describe())
df_data.columns

df_test.describe()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


               store           item          sales
count  913000.000000  913000.000000  913000.000000
mean        5.500000      25.500000      52.250287
std         2.872283      14.430878      28.801144
min         1.000000       1.000000       0.000000
25%         3.000000      13.000000      30.000000
50%         5.500000      25.500000      47.000000
75%         8.000000      38.000000      70.000000
max        10.000000      50.000000     231.000000


,id,store,item
count,45000.000000,45000.000000,45000.00000
mean,22499.500000,5.500000,25.50000
std,12990.525394,2.872313,14.43103
min,0.000000,1.000000,1.00000
25%,11249.750000,3.000000,13.00000
50%,22499.500000,5.500000,25.50000
75%,33749.250000,8.000000,38.00000
max,44999.000000,10.000000,50.00000


In [3]:
if env == "test":
    # Take the last 3 months of 2017 as testing data
    df_test = df_data[df_data.date >= '2017-10-01']
    # Remove the last 3 months, as it would not be fair to train on those
    df_train = df_data[df_data.date < '2017-10-01']
else:
    df_train = df_data
    df_test['sales'] = 0

# Only select a small sample, faster local testing
if sample_size is not None and sample_size > 0:
    df_train = df_train.sample(sample_size)
    
display(df_train.head(5))
df_test.describe()

,date,store,item,sales
93942,2015-03-28,2,6,79
424072,2014-03-17,3,24,55
644811,2013-08-22,4,36,64
236311,2015-01-28,10,13,59
257775,2013-11-06,2,15,81


,store,item,sales
count,46000.000000,46000.000000,46000.000000
mean,5.500000,25.500000,54.677543
std,2.872313,14.431027,28.485947
min,1.000000,1.000000,3.000000
25%,3.000000,13.000000,32.000000
50%,5.500000,25.500000,50.000000
75%,8.000000,38.000000,73.000000
max,10.000000,50.000000,174.000000


In [4]:
# We add all data sets to add dummies there, and every relevant columns
def prepare_data(df1, df2):
    count1 = len(df1)
    count2 = len(df2)

    print("count1: {0}".format(count1))
    print("count2: {0}".format(count2))
    df = df1.append(df2, ignore_index=True)
    # Add time column, easier for later step
    df = df.assign(time = df.date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d")))
    
    # nb days since the beginning of the data, as traffic grows by store
    df = df.assign(days = df.time.apply(lambda x: (x - datetime(2012,12,31)).days))
    
    # Week day should be used 
    df = df.assign(weekday = df.time.apply(lambda x: x.weekday()))
    
    df = df.assign(dom = df.time.apply(lambda x: x.day))
    
    df = df.assign(cw = df.time.apply(lambda x: x.isocalendar()[1]))

    df = df.assign(month = df.time.apply(lambda x: x.month))
    
    df = pandas.get_dummies(df, prefix=['store', 'item', 'dom', 'cw', 'weekday', 'month'], 
                            columns=['store', 'item', 'dom', 'cw', 'weekday', 'month'])
    
    # Return both dataframe, by number
    return df[:count1], df[count1:]

df_train_prepared, df_test_prepared = prepare_data(df_train, df_test)
print("df_train1 len: {0}, df_test1 len: {1}".format(len(df_train_prepared), len(df_test_prepared)))
display(df_train_prepared.head(5))
display(df_test_prepared.head(5))

count1: 40000
count2: 46000
df_train1 len: 40000, df_test1 len: 46000


,date,sales,time,days,store_1,store_2,store_3,store_4,store_5,store_6,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2015-03-28,79,2015-03-28,817,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2014-03-17,55,2014-03-17,441,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2013-08-22,64,2013-08-22,234,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,2015-01-28,59,2015-01-28,758,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2013-11-06,81,2013-11-06,310,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


,date,sales,time,days,store_1,store_2,store_3,store_4,store_5,store_6,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
40000,2017-10-01,21,2017-10-01,1735,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
40001,2017-10-02,12,2017-10-02,1736,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
40002,2017-10-03,18,2017-10-03,1737,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
40003,2017-10-04,15,2017-10-04,1738,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
40004,2017-10-05,20,2017-10-05,1739,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# Generate our training/validation datasets
from sklearn import model_selection

# Name of the result column
result_cols = ['sales']
result_excl_cols = 'sales_'
# Removing input_cols = ['store', 'item',
# dom, cw, 

# best model contained:
# days, store, item, weekday, month, cw, dom
input_cols = [
#    'sales_',
    'store_',
    'item_',
    #'day', always out
    #'day_',
    #'weekday', always out
    #'dom_',
    'cw_',
    'weekday_',
    #'month', always out
    #'month_',
    
    'days'
]

# Get the final values
def get_values(df, cols=[], excl_cols = "doqwidjoqwidjqwoidjqwoidjqwodijqw"):
    columns = df.columns.values
    # Remove all columns that are not inside the list
    for column in columns:
        find = False
        if column.startswith(excl_cols):
            print("Ignoring {0}".format(column))
        else:
            for col in cols:
                if column.startswith(col):
                    find = True
        if not find:
            df = df.drop(columns=[column])
    new_order = sorted(df.columns.values)
    print(new_order)
    # Same order for both training and testing set
    df = df[new_order]
    return df.values

X_train = get_values(df_train_prepared, input_cols)
Y_train = get_values(df_train_prepared, result_cols, result_excl_cols).ravel()
X_test = get_values(df_test_prepared, input_cols)

# In test env, we calculate it for the test only
if env == "test":
    Y_test = get_values(df_test_prepared, result_cols, result_excl_cols).ravel()

['cw_1', 'cw_10', 'cw_11', 'cw_12', 'cw_13', 'cw_14', 'cw_15', 'cw_16', 'cw_17', 'cw_18', 'cw_19', 'cw_2', 'cw_20', 'cw_21', 'cw_22', 'cw_23', 'cw_24', 'cw_25', 'cw_26', 'cw_27', 'cw_28', 'cw_29', 'cw_3', 'cw_30', 'cw_31', 'cw_32', 'cw_33', 'cw_34', 'cw_35', 'cw_36', 'cw_37', 'cw_38', 'cw_39', 'cw_4', 'cw_40', 'cw_41', 'cw_42', 'cw_43', 'cw_44', 'cw_45', 'cw_46', 'cw_47', 'cw_48', 'cw_49', 'cw_5', 'cw_50', 'cw_51', 'cw_52', 'cw_53', 'cw_6', 'cw_7', 'cw_8', 'cw_9', 'days', 'item_1', 'item_10', 'item_11', 'item_12', 'item_13', 'item_14', 'item_15', 'item_16', 'item_17', 'item_18', 'item_19', 'item_2', 'item_20', 'item_21', 'item_22', 'item_23', 'item_24', 'item_25', 'item_26', 'item_27', 'item_28', 'item_29', 'item_3', 'item_30', 'item_31', 'item_32', 'item_33', 'item_34', 'item_35', 'item_36', 'item_37', 'item_38', 'item_39', 'item_4', 'item_40', 'item_41', 'item_42', 'item_43', 'item_44', 'item_45', 'item_46', 'item_47', 'item_48', 'item_49', 'item_5', 'item_50', 'item_6', 'item_7', 'i

In [6]:
df_test_prepared.describe()

,sales,days,store_1,store_2,store_3,store_4,store_5,store_6,store_7,store_8,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
count,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,46000.000000,...,46000.0,46000.0,46000.0,46000.0,46000.0,46000.0,46000.0,46000.000000,46000.000000,46000.000000
mean,54.677543,1780.500000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.336957,0.326087,0.336957
std,28.485947,26.556832,0.300003,0.300003,0.300003,0.300003,0.300003,0.300003,0.300003,0.300003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.472675,0.468785,0.472675
min,3.000000,1735.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,32.000000,1757.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
50%,50.000000,1780.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
75%,73.000000,1803.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000
max,174.000000,1826.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000


In [7]:
# Normalize the data
X_all = [x + y for x, y in zip(X_train, X_test)]
#print(len(X_all))
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler() 

# Don't cheat - fit only on training data
# Def adding x_train + X_test + X_validation to fit all of them

scaler.fit(X_train)  

X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
# Custom function to calculate the SMAPE
def get_smape(Y_validation, Y_validation_predict):
    result = 0
    for i in range(0, len(Y_validation)):
        result += (abs(Y_validation[i] - Y_validation_predict[i]))/(abs(Y_validation[i])+abs(Y_validation_predict[i]))
    return result / len(Y_validation) * 200

In [9]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier


models = []

#models.append(("LogisticRegression",LogisticRegression()))
#models.append(("SVC",SVC()))
#models.append(("LinearSVC",LinearSVC()))
#models.append(("KNeighbors",KNeighborsClassifier()))
#models.append(("DecisionTree",DecisionTreeClassifier()))
#models.append(("RandomForest",RandomForestClassifier()))

models.append(('MLPRegressor_adam', MLPRegressor(hidden_layer_sizes=(8,),  activation='relu', solver='adam',
                                                               alpha=0.0001, batch_size='auto',
learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=1000, shuffle=True,
random_state=2, tol=0.0001, verbose=False, warm_start=False, nesterovs_momentum=True,
beta_1=0.9, beta_2=0.999, epsilon=1e-08)))
    

# High value until first model get solved
max_score = 10000
best_model = "UNKNOWN"

res = []
# Testing all models, one by one
for name, model in models:
    print("Executing for model {0}".format(name))
    time_start = datetime.now()

    # Training the model
    model.fit(X_train, Y_train)
    
    print("Finish fit for {0}".format(name))

    Y_test_result = model.predict(X_test)
    res.append(Y_test_result)
    if env == "test":
        # We can calculate the avg error
        score = get_smape(Y_test, Y_test_result)
        print("Model {0} got score of {1}, time: {2}".format(name, score, datetime.now() - time_start))
    else:
        # Let's write an output file, with the name of the model
        print("Writing output file {0}.csv for model {0}".format(name))
        
        df_test['sales'] = Y_test_result
        result_df = df_test[['id', 'sales']]
        result_df['sales'] = Y_test_result
        
        result_df.to_csv("{0}.csv".format(name), index=False)

Executing for model MLPRegressor_adam
Finish fit for MLPRegressor_adam
Model MLPRegressor_adam got score of 13.1224797853156, time: 0:00:21.951016
